# Imports

In [ ]:
# From Imports
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
from pandas import DataFrame
from datetime import datetime
from scipy.optimize import minimize
from arch import arch_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Alias Imports
import numpy_financial as npf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pmdarima as pm

# Imports
import math
import arch
import openpyxl
import pprint

print("Import Complete")

Import Complete


# Importing Uniswap Pool, LP, and USDC Lending Rate Data

In [ ]:
prices_100bp = []
volumes_100bp = [] #total $USD volume of USDC and ETH traded on each block
avg_LP_spread_100bp = [] #probably need an intermediate step to calc this, avg_LP_upper - avg_LP_lower on each block interval
active_liq_depth_100bp = [] # sum(x) * sum (y), where the sums are based on the LP positions that contain the current price in their range


# Predicted Loss Function Calculations

In [ ]:
#Helper functions

#Used to calculate alpha_lu, alpha_d, and alpha_0
def alpha_t(k, Z, Zl, Zu)
        if Zl < Z < Zu
            xi = k * (Z**(1/2) - (Zl)**(1/2))
            yi = k * (Z**(-1/2) - Zu**(-1/2))
            alpha = xi + yi * Z
        elif Z < Zl:
            xi = 0
            yi = k * ((Zl)**(-1/2) - (Zu)**(-1/2))
            alpha = xi + yi * Z
        elif Z > Zu:
            xi = k * ((Zu)**(1/2) - (Zl)**(1/2))
            yi = 0
            alpha = xi + yi * Z
        return alpha

#See denominator or convexity cost
def LP_spread_conversion(Z, Zl, Zu):
    sigma_u = -2 * (Z**(1/2)) / (Zu**(1/2)) + 2
    sigma_l = -2 * (Z**(1/2)) / (Zl**(1/2)) + 2
    return sigma_u + sigma_l

#Quadratic Variation, needs multiple step ahead forecast, maybe generate 20 forecasts over the short block interval
def calc_QV(Z):
    log_returns = np.diff(np.log(price_list))
    QV = []
    for i in range(10, len(log_returns), 10):
        QV.append(np.sum(np.diff(log_returns[i-10:i])**2))
    QV = pd.Series(QV)
    return QV


In [ ]:
#Combining calculations
#functionally very similar to LVR



#Active tick range data
alpha_lu = []
Zl_avg = [] #for every single timestamp, this list contains the average lower bound of the LPs
Zu_avg = [] #for every single timestamp, this list contains the average upper bound of the LPs


def calculate_conv_cost(alpha_lu, Zl, Zu, Z):
    integral = 0
    for i in range(len(Z)):
      alpha_curr_lu = alpha_lu[i]
      spread = LP_spread_conversion(Z[i], Zl_avg[i], Zu_avg[i])
      quad_var = calc_QV(Z[:i])
      integral += alpha_curr_lu / (2* spread * Z[i]**2) * quad_var

    return integral

convexity_cost = calculate_conv_cost()

In [ ]:
def calculate_opp_cost(alpha_lu, initial_alpha, r):
    integral = 0
    alpha_d = initial_alpha

    for i in range(len(alpha_lu)):
        integral += (alpha_lu[i] - alpha_0) * r[i]
    return integral

initial_alpha = alpha_lu[0]

opportunity_cost = calculate_opp_cost(Z_values, r_values, k, Zl, Zu, initial_alpha)

In [ ]:
#PL = convexity_cost + opportunity_cost

# Fitting and Forecasting Parameters using TS Models

In [ ]:
#NOTE, EDITED USING ROLLING ORIGIN VALIDATION (ROV)

# Define the number of initial observations for training
training_size = 100
# Define the size of the rolling testing window
testing_size = 1

# Initialize the ARIMA-GARCH model
arima_model = pm.auto_arima(Z[:training_size], suppress_warnings=True)
garch_model = arch.arch_model(Z[:training_size], vol='Garch', p=1, q=1)

# Lists to store performance metrics
mae_list = []
mse_list = []

# Iterate through the data using ROCV
for i in range(training_size, len(Z) - testing_size + 1):
    training_data = Z[i - training_size:i]
    testing_data = Z[i:i + testing_size]

    # Re-estimate the ARIMA model with new training data
    arima_model.fit(training_data)
    best_arima_order = arima_model.order

    # Re-estimate the GARCH model with new training data
    garch_model = arch.arch_model(training_data, vol='Garch', p=best_arima_order[0], q=best_arima_order[1])
    res = garch_model.fit(disp='off')

    # Forecast the next value using the updated model
    forecast = arima_model.predict(n_periods=testing_size)
    volatility = res.conditional_volatility[-1]
    forecast = forecast * np.sqrt(volatility)

    # Calculate evaluation metrics
    mae = mean_absolute_error(testing_data, forecast)
    mse = mean_squared_error(testing_data, forecast)

    mae_list.append(mae)
    mse_list.append(mse)

# Calculate the average or other statistics for the performance metrics
average_mae = np.mean(mae_list)
average_mse = np.mean(mse_list)



In [ ]:
#Asks

#1. Pull following data for 3 pools and put into list format: prices, alpha_lu (aggregate at each block stamp, not individual), Zl_avg (average lower tick / block)
# Zu_avg (average upper tick per block)

#2. Plug each of those into the code above where the list Z is being utilized, see how long runtime is

#3. Modify code to store forecasts at one step ahead intervals

#4. Plug those parameters into PL code and print results

#5. Use volume series, again forecast and save one step ahead forecasts

#6. Divide PL / Volume series to determine a fee %, plot fee percent over time to sanity check
